In [ ]:
import azureml.core
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep

print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="AutoML-SDK")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Compute target exists')
except ComputeTargetException:
    print('Creating a new compute target')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
ws.datasets.keys()

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Houses"
description_text = "House prices and characteristics."

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        from azureml.core import Workspace, Dataset

        subscription_id = 'aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee'
        resource_group = 'aml-quickstarts-137127'
        workspace_name = 'quick-starts-ws-137127'

        workspace = Workspace(subscription_id, resource_group, workspace_name)

        dataset = Dataset.get_by_name(workspace, name='Houses')

df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
dataset.take(5).to_pandas_dataframe()

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric": 'spearman_correlation'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="Transactieprijs_m2",   
                             path = project_folder,
                             validation_size = 0.2,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
run = exp.submit(config=automl_config, show_output=True)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.
best_run, best_model = run.get_output()

In [ ]:
print(best_run)

In [ ]:
print(best_model)

In [ ]:
best_run.register_model(model_name = "automl_udacity_project.pkl", model_path = './outputs/')
print(best_model._final_estimator)

In [ ]:
compute_target.delete()